In [6]:
#!python

from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Activation, LSTM, Bidirectional
from tensorflow.python.keras import metrics
from helper_funct import *
import time
import os

# TODO: gradient back propagation
#       freeze weights and load them for more layers. Train stack
#       use 100 units to find best method.
#       Camelyon17 images. Use CNN network
#       combine image and MSI data. Image captioning
#       stateful = true, or return sequences in last layer



In [10]:

# HYPER Parameters
mini_batch_size = 32
embedding_size = 8
learning_rate = 0.005
epoch_train = 300  # maximum repetitions
validation_split = 0.05
optimizer_str = "RMSprop"
optimizer = keras.optimizers.RMSprop(lr=learning_rate, rho=0.9, epsilon=None, decay=0.0) # keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True) #
metrics = ['accuracy', 'mae']
bias_init = 'he_normal'  # It draws samples from a truncated normal distribution centered on 0 with stddev = sqrt(2 / fan_in) where  fan_in is the number of input units in the weight tensor.
kernel_init = 'he_normal'
weight_init = 'he_normal'
use_bias = True
verbose = 2

# Model parameters
units = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
layers = [1, 2, 3, 5, 8, 10]
lstm_type = ['LSTM', 'Bidirectional']
activation = 'softmax'
loss_function = 'binary_crossentropy'
merge_mode = ['average', 'concat', 'max']

units = [50]
layers = [2, 3, 4, 5, 6, 7, 8, 9, 10]
lstm_type = ['LSTM']
merge_mode = 'average'

file = open("results.txt", "a")
file.write("mini_batch_size: {} learning_rate: {} optimizer: {}".format(mini_batch_size, learning_rate, optimizer_str))


# Data specific parameters
n_sets = 8
n_folds = 4
num_classes = 2
time_steps = 1


In [11]:
# initialing the output array
bal_accuracy = np.zeros(n_folds)
balanced_accuracy = np.zeros((len(units), len(layers), len(lstm_type), n_folds))
run_time2 = np.zeros((len(units), len(layers), len(lstm_type)))

# Reading the data from the read_data function
x_train, x_test, x_val, y_val, y_train, y_test = read_data(n_folds)

# in_shape defines the input shape of the LSTM modules
in_shape = len(x_train[0][0][0])  # data length variable for the input tensor


In [12]:

start_time = time.time()
for unit in units:
    for n_layers in layers:
        for type in lstm_type:
            print "unit: ", unit
            print "n_layers: ", n_layers
            print "type: ", type
            start_time2 = time.time()
            model = Sequential()
            # Adding the LSTM layers or the Bidirectional LSTM modules
            if type == 'LSTM':
                if n_layers > 1:
                    model.add(LSTM(units=unit, batch_input_shape=(mini_batch_size, time_steps, in_shape), return_sequences=True, stateful=True,
                                   bias_initializer=bias_init, kernel_initializer=kernel_init,
                                   recurrent_initializer=weight_init, use_bias=use_bias, name="LSTM_1"))
                    for j in range(2, n_layers):
                        model.add(LSTM(units=unit, return_sequences=True, stateful=True, bias_initializer=bias_init,
                                       kernel_initializer=kernel_init, recurrent_initializer=weight_init,
                                       use_bias=use_bias, name = "LSTM_{}".format(j)))
                    model.add(LSTM(units=unit, bias_initializer=bias_init, kernel_initializer=kernel_init,
                                   recurrent_initializer=weight_init, use_bias=use_bias, name="LSTM_{}".format(n_layers)))
                else:
                    model.add(LSTM(units=unit, input_shape=(time_steps, in_shape), bias_initializer=bias_init,
                                   kernel_initializer=kernel_init, recurrent_initializer=weight_init,
                                   use_bias=use_bias, name="LSTM_1"))
            else:
                if n_layers > 1:
                    model.add(Bidirectional(LSTM(units=unit, return_sequences=True, stateful=True, bias_initializer=bias_init, kernel_initializer=kernel_init, 
                                                 recurrent_initializer=weight_init, use_Bias=use_bias), batch_input_shape=(mini_batch_size, time_steps, in_shape), 
                                            merge_mode=merge_mode))
                    for j in range(1, n_layers-1):
                        model.add(Bidirectional(LSTM(units=unit, return_sequences=True, stateful=True, bias_initializer=bias_init, kernel_initializer=kernel_init, 
                                                     recurrent_initializer=weight_init, use_Bias=use_bias), merge_mode=merge_mode))
                    model.add(Bidirectional(LSTM(units=unit, bias_initializer=bias_init, kernel_initializer=kernel_init, recurrent_initializer=weight_init, 
                                                 use_Bias=use_bias), merge_mode=merge_mode))
                else:
                    model.add(Bidirectional(LSTM(units=unit, bias_initializer=bias_init, kernel_initializer=kernel_init, recurrent_initializer=weight_init, 
                                                 use_Bias=use_bias), input_shape=(time_steps, in_shape), merge_mode=merge_mode))

            models_recovering = [Sequential() for _ in xrange(n_folds)]
            i = n_layers - 1
            layers_to_load = 0
            isloaded = False
            while i > 0 and not isloaded:
                file_name = "Models_layers:{}_Type:{}_units:{}_Set:0.h5".format(i, type, unit)
                if os.path.isfile(file_name):
                    print "loading: {}".format(file_name)
                    for k in xrange(n_folds):
                        models_recovering[k].load_weights("Model_layers:{}_Type:{}_units:{}_Set:{}.h5".format(i, lstm_type, units, k), by_name=True)
                    isloaded = True
                    layers_to_load = i
                i -= 1
            if isloaded:
                for layer in model.layers[:i+1]:
                    layer.trainable = False

            for layer in model.layers:
                print(layer, layer.trainable)
                print(layer.name)

            # Adding the rest of the network's components
            model.add(Dense(units=num_classes))
            model.add(Activation(activation=activation))
            # model.compile(loss=loss_function, optimizer=optimizer, metrics=metrics)
            model.summary()

            # training of the model
            bal_accuracy = train_model(x_train, y_train, x_val, y_val, validation_split, epoch_train, mini_batch_size,
                                            x_test, y_test, model, n_folds, learning_rate, optimizer, verbose,
                                            loss_function, metrics, isloaded, n_layers, layers_to_load, type, unit)

            # Saving the balanced accuracy over the 4 folds
            balanced_accuracy[units.index(unit), layers.index(n_layers), lstm_type.index(type)] = bal_accuracy
            run_time2[units.index(unit), layers.index(n_layers), lstm_type.index(type)] = time.time() - start_time2
            print "run time of units {} n_layers {} of type {} : {}\n\n---------------------\n\n".format(unit, n_layers, type, run_time2[units.index(unit), layers.index(n_layers), lstm_type.index(type)])

            file.write(" units: {} n_layers: {} type: {} balanced accuracy: {}\n".format(unit, n_layers, type, bal_accuracy))

            del model

run_time = time.time() - start_time


unit:  50
n_layers:  1
type:  LSTM
(<tensorflow.python.keras._impl.keras.layers.recurrent.LSTM object at 0x7fc986c1ff50>, True)
LSTM_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_1 (LSTM)                (None, 50)                944400    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 102       
_________________________________________________________________
activation_4 (Activation)    (None, 2)                 0         
Total params: 944,502
Trainable params: 944,502
Non-trainable params: 0
_________________________________________________________________


Epoch 1/300


 - 4s - loss: 0.6897 - acc: 0.5641 - mean_absolute_error: 0.4896


Epoch 2/300


 - 3s - loss: 0.6762 - acc: 0.5829 - mean_absolute_error: 0.4810


...

Epoch 176/300


 - 3s - loss: 0.1675 - acc: 0.9259 - mean_absolute_error: 0.0930


Epoch 177/300


 - 3s - loss: 0.1660 - acc: 0.9279 - mean_absolute_error: 0.0887


Epoch 178/300


In [1]:

print ""
print "balanced_accuracy: "
print balanced_accuracy
print "run_time2: "
print run_time2
# print ""

print "Total run time: ", run_time

balanced_accuracy_final = balanced_accuracy.mean()

print("")
print "Successfully trained and run with balanced_accuracy_final: ", balanced_accuracy_final

file.write("time: {}\n\n".format(run_time))
file.close()



balanced_accuracy: 


NameError: name 'balanced_accuracy' is not defined